# Импорт библиотек

In [1]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import chromadb
import torch
import os
from google.colab import drive
from transformers import AutoModelForQuestionAnswering
from transformers import AutoTokenizer
from transformers import pipeline

In [2]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cuda


## Подключение Google Drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


# Подготовка хранилища данных

In [5]:
# Удобная функция для получения модели эмбеддингов
def get_embeddings(model_name):
    model_kwargs = {'device': DEVICE}
    return HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs
    )

In [6]:
# Загрузка БД, созданной в chroma_preparing.ipynb
db = Chroma(persist_directory='/content/drive/MyDrive/M_QA/chroma/war_and_peace',
            collection_name='war_and_peace',
            collection_metadata={'hnsw:space': 'cosine'},
            embedding_function=get_embeddings('d0rj/e5-base-en-ru'))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/471 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Подготовка модели для ответов

In [12]:
model_name = '/content/drive/MyDrive/M_QA/ruBERT_QA_1_epoch'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(DEVICE)
qa_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)

In [13]:
# Пример ответов модели ruBERT-base
qa_pipeline(question=['Кто я?', 'Что я планирую?'],
            context=['Я был рожден не каким-то там человеком, а ящером, и теперь я готов захватить мир']*2)

[{'score': 0.16905491054058075,
  'start': 0,
  'end': 48,
  'answer': 'Я был рожден не каким-то там человеком, а ящером'},
 {'score': 0.4450800120830536,
  'start': 67,
  'end': 80,
  'answer': 'захватить мир'}]

# Объединенная система

In [14]:
# Проверить, является ли объект итеративным
def is_iterable(obj):
    try:
        iter(obj)
        return not isinstance(obj, (str, dict))
    except TypeError:
        return False

In [15]:
# Извлечь название книги из его пути
def extract_book_name(path):
    base_name = os.path.basename(path)
    return os.path.splitext(base_name)[0]

In [16]:
# Функция для адекватного вывода текстов фрагментов
def wrap_text(text, width=100, indent=''):
    text = text.replace('\n', ' ').replace('\r', '')
    lines = [indent + text[i:i+width] for i in range(0, len(text), width)]
    return '\n'.join(lines)

In [20]:
# Главная функция объединенной QA-системы
def qa_system(questions, k_relevant=3, return_type=0):
    """
    Главная функция для взаимодействия с QA-системой

    PARAMETERS
    ----------
    questions : str or list[str]
        вопрос или список вопросов к системе
    
    k_relevent : int
        количество релевантных фрагментов к каждому
        вопросу
    
    return_type : 0 or 1 or 2
        формат вывода ответов:
        - 0: вернуть только список ответов;
        - 1: вернуть кортеж из списка ответов, списка
             соответствующих источников и списка абсолютных
             позиций ответов в этих источниках;
        - 2: вернуть список из кортежей (контекст, источник),
             где в контексте выделен ответ, а в источнике
             указан и документ, и позиция ответа в нем
    """
    # Преобразование вопроса в список вопросов
    if not is_iterable(questions):
        questions = [questions]

    # Поиск релевантных отрывков для каждого вопроса
    contexts_data = [db.similarity_search_with_score('query: ' + q,
                                                     k=k_relevant)
                     for q in questions]

    # Обрезка префикса "passage: " у контекстов
    for contexts_with_score in contexts_data:
        for context, _ in contexts_with_score:
            context.page_content = context.page_content[9:]

    # Список из собранных контекстов для каждого вопроса
    contexts_texts = ['\n'.join(context.page_content#[9:]
                               for context, score in k_contexts)
                     for k_contexts in contexts_data]

    # Получение ответов с помощью модели
    answers_data = qa_pipeline(question=questions,
                               context=contexts_texts)

    # Преобразование ответов в список
    if not is_iterable(answers_data):
        answers_data = [answers_data]

    # Извлечение только текстов ответа
    answers_texts = [answer['answer'] for answer in answers_data]

    # Вернуть тексты ответов
    if return_type == 0:
        return answers_texts

    # Цвета для выделения ответа в контексте
    selection_start = '\033[96m'
    selection_end = '\033[0m'

    context_ids = []
    sources = []
    rel_source_starts = []
    abs_source_starts = []

    # Определение позиции ответов в контексте
    for i, (context_data, answer_data) in enumerate(zip(contexts_data,
                                                   answers_data)):
        cumulative_starts = 0
        for j, (context_doc, score) in enumerate(context_data):
            if answer_data['answer'] in context_doc.page_content:
                context_ids.append(j)
                sources.append(extract_book_name(context_doc.metadata['source']))
                rel_source_starts.append(answer_data['start'] - cumulative_starts)
                abs_source_starts.append(rel_source_starts[-1] + context_doc.metadata['start_index'])
                break
            cumulative_starts += len(contexts_data[i][j][0].page_content) + 1

    if return_type == 1:
        return answers_texts, sources, abs_source_starts
    else:
        selected_answers = []
        for i in range(len(questions)):
            # Обрезание префикса "passage: " у контекста
            context = contexts_data[i][context_ids[i]][0].page_content
            start_pos = rel_source_starts[i]
            end_pos = start_pos + len(answers_data[i]['answer'])
            selected_answers.append((''.join([context[:start_pos],
                                              selection_start,
                                              context[start_pos:end_pos],
                                              selection_end,
                                              context[end_pos:]]),
                                     f'(C) {sources[i]}, символ {abs_source_starts[i]}'))

        return selected_answers

In [30]:
# Вывести ответ системы, если использовался return_type=2
def print_qa_result(result, width=100):
    text, source = result
    text = text.replace('\r', '')
    print(wrap_text(text, width=width))
    print(f'{source:>{width}}')

# Примеры работы системы

In [22]:
%%time

questions = [
    'Когда было получено известие о потере Москвы?',
    'В каком году было Бородинское сражение?',
    'Когда было Бородинское сражение?',
    'Где служил Николай Ростов?',
    'В каком городе была главная квартира Кутузова?',
    'Когда русские войска располагались у крепости Бранау?',
    'Где смертельно ранили Андрея Болконского?'
]

qa_results = qa_system(questions, k_relevant=3, return_type=3)
for question, result in zip(questions, qa_results):
    print('Вопрос:', question)
    print('В контексте:')
    print_qa_result(result)
    print()

Вопрос: Когда было получено известие о потере Москвы?
В контексте:
о Бородинском сражении, о наших потерях убитыми и ранеными, а еще более страшное известие о потере Москвы были получены в Воронеже в половине сентября. Княжна Марья, узнав только из газет о ране брата и не имея о нем никаких определенных сведений, собралась ехать отыскивать князя Андрея, как слышал Николай (сам же он не видал ее). Получив известие о Бородинском сражении и об оставлении Москвы, Ростов не то чтобы испытывал отчаяние, злобу или месть и тому подобные чувства, но ему вдруг все стало
                                                               (C) Война и мир. Том 4, символ 47994

Вопрос: Когда было Бородинское сражение?
В контексте:
пущено ни одного выстрела ни с той, ни с другой стороны, 26-го произошло Бородинское сражение. Для чего и как были даны и приняты сражения при Шевардине и при Бородине? Для чего было дано Бородинское сражение? Ни для французов, ни для русских оно не имело ни малейшего смысла. 